# Demultiplexing the MultiSpecies Comparison data.

### Timeline:
1. FASTQ files aligned to mixed reference genome using ParseNIP. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2</span>
2. Modifying BAM files: CB tag in BAM files for each sublibrary (10 sublibraries) was surogated for a new 16-bp sequence analogous to 10X barcodes to bridge the incompatibility between cellbouncer and Parse Biosciences. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/modified_bam</span>
3. Parse–sudo10X translation dictionaries derived from the translated files. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT*_barcodes.csv</span>
4. Translated BAM files processed with utils/composite_bam2counts -b [bamfile] -o [output_directory]. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts</span>
5. Outputs processed with demux_species –o [output_directory] to get summary of species assigned for each cell barcodes (sudo10X) , the confidence and singlet/doublet score.
6. Created a metadata with parse barcodes, sudo10x barcode, singlet/doublet score, confidence, species assigned. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed/metadata_barcode_species_ParseWT*.csv</span>
7. Created list of barcodes of each species for each sublibrary. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed/ParseWT__barcodes.csv</span>
8. Filtering FASTQ files into separate species. \
8.1 Making lists of headers corresponding to each species per sublibarary. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed/Fastq_filters</span> \
8.2 Concatanting FASTQ of each sublibrary across sequencing lanes. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_merged</span> \
8.3 Filtering FASTQ files into separate species. <span style="color:#87CEEB; font-family:'Consolas', 'Menlo', 'DejaVu Sans Mono', monospace; font-weight:500">/cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered</span>
9. Realignment of species FASTQ to species-specific reference genomes using ParseNIP. 
10. CrossFilt removal of annotation biases from resulting BAM files: crossfilt-lift.
11. Turning BAM files into counts matricies. 

#### 1. FASTQ file aligned to mixed regerence genome. 

In [ ]:
#mkaing mixed reference genome 
nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
    -profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
    --fasta Ensembl/homo_sapiens/GRCh38/Release_102/FASTA/homo_sapiens__GRCh38__release102.dna.fa,Ensembl/pan_troglodytes/Pan_tro_3.0/Release_105/FASTA/pan_troglodytes__Pan_tro_3.0__release105.dna.fa,Ensembl/gorilla_gorilla/gorGor4/Release_110/FASTA/gorilla_gorilla__gorGor4__release110.dna.fa,test_genome/Macaca_fascicularis_test.fa,Ensembl/mus_musculus/GRCm39/Release_108/FASTA/mus_musculus__GRCm39__release108.dna.fa \
    --gtf Ensembl/homo_sapiens/GRCh38/Release_102/GTF/Homo_sapiens.GRCh38.102.gtf,Ensembl/pan_troglodytes/Pan_tro_3.0/Release_105/GTF/Pan_troglodytes.Pan_tro_3.0.105.gtf,Ensembl/gorilla_gorilla/gorGor4/Release_110/GTF/Gorilla_gorilla.gorGor4.110.gtf,Ensembl/macaca_fascicularis/Macaca_fascicularis_6.0/Release_112/GTF/Macaca_fascicularis.Macaca_fascicularis_6.0.112.gtf,Ensembl/mus_musculus/GRCm39/Release_108/GTF/Mus_musculus.GRCm39.108.gtf \
    --genome_name GRCh38,Pan_tro_3.0,gorGor4,Macaca_fascicularis_6.0,GRCm39 -bg

#alignment with ParseNIP
nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
-profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
--genome_dir /cephfs2/hannas/MultiSpeciesComp/final/genome/mixed_ref_genome/results/GENOME_INDEX \
--samp_list /cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/SampleLoadingTable_MultiSpieciesComp.txt \
--fastq /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq --chemistry v1 --concatenate -bg


#### 2. Modifying BAM files

In [ ]:
import pysam
import random

# Your exact functions (unchanged)
def extract_unique_barcodes_from_bam(bam_file_path, output_file="unique_barcodes_subset1000.txt"):
    unique_barcodes = set()
    with pysam.AlignmentFile(bam_file_path, "rb") as bam_file:
        for read in bam_file:
            if read.has_tag('CB'):
                cb_tag = read.get_tag('CB')
                unique_barcodes.add(cb_tag)
        unique_barcode_list = sorted(list(unique_barcodes))
        with open(output_file, 'w') as f:
            for barcode in unique_barcode_list:
                f.write(f"{barcode}\n")
        return unique_barcode_list

def generate_unique_16bp_sequences(num_sequences):
    unique_sequences = set()
    while len(unique_sequences) < num_sequences:
        sequence = ''.join(random.choice('ACGT') for _ in range(16))
        unique_sequences.add(sequence)
    return list(unique_sequences)

def new_barcodes_bam(input_bam, output_bam, translation_dict):
    with pysam.AlignmentFile(input_bam, "rb") as input_file:
        with pysam.AlignmentFile(output_bam, "wb", header=input_file.header) as output_file:
            for read in input_file:
                if read.has_tag('CB'):
                    old_barcode = read.get_tag('CB')
                    if old_barcode in translation_dict:
                        new_barcode = translation_dict[old_barcode]
                        read.set_tag('CB', new_barcode, 'Z')
                output_file.write(read)

# Pipeline
bam_path = "/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/subset_1000.bam"

unique_barcode_list = extract_unique_barcodes_from_bam(bam_path)
new_sequences = generate_unique_16bp_sequences(len(unique_barcode_list))
translation_dict = dict(zip(unique_barcode_list, new_sequences))
new_barcodes_bam(bam_path, "translated_output.bam", translation_dict)

# List of BAM files
# List of BAM files
bam_files = [
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT21.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT22.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT33.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT34.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT35.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT36.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT37.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT38.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT39.22K75GLT4/process/barcode_headAligned_anno.bam",
    "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/results/splitpipe/SLX-25498.ParseWT40.22K75GLT4/process/barcode_headAligned_anno.bam"
]

# Core names for output files
output_names = [
    "ParseWT21",
    "ParseWT22",
    "ParseWT33",
    "ParseWT34",
    "ParseWT35",
    "ParseWT36",
    "ParseWT37",
    "ParseWT38",
    "ParseWT39",
    "ParseWT40"
]
# Process each file
for i, bam_path in enumerate(bam_files):
    core_name = output_names[i]
    
    unique_barcode_list = extract_unique_barcodes_from_bam(bam_path, f"unique_barcodes_{core_name}.txt")
    new_sequences = generate_unique_16bp_sequences(len(unique_barcode_list))
    translation_dict = dict(zip(unique_barcode_list, new_sequences))
    new_barcodes_bam(bam_path, f"translated_{core_name}.bam", translation_dict)

### 3. Parse–sudo10X translation dictionaries derived from the translated files.

In [ ]:
import csv
import pysam
import glob
import os

# Find all BAM files
bam_files = glob.glob("/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/modified_bam/*.bam")

print(f"Found {len(bam_files)} BAM files")

# Process each BAM file
for bam_file in bam_files:
    print(f"Processing {os.path.basename(bam_file)}...")
    
    # Create dictionary for this file
    cb_dict = {}
    with pysam.AlignmentFile(bam_file, "rb") as bam:
        for read in bam:
            if read.has_tag("CB"):
                cb_tag = read.get_tag("CB")
                if cb_tag not in cb_dict:
                    cb_dict[cb_tag] = read.query_name[:8]
    
    # Save as CSV
    base_name = os.path.splitext(os.path.basename(bam_file))[0]
    output_csv = f"{base_name}_barcodes.csv"
    
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['cell_barcode', 'read_name'])
        for cb, read_name in cb_dict.items():
            writer.writerow([cb, read_name])
    
    print(f"  Saved {len(cb_dict)} barcodes to {output_csv}")

print("Done!")

### 4. Translated BAM files processed with utils/composite_bam2counts


In [ ]:
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT21.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT22.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT22
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT33.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT33
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT34.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT34
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT35.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT35
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT36.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT36
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT37.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT37
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT38.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT38
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT39.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT39
utils/composite_bam2counts -b/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_ParseWT40.bam -o /cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/ParseWT40

### 5. Outputs processed with demux_species –o [output_directory]

### 6. Created a metadata with parse barcodes, sudo10X barcodes, species assigned, singlet/doublet score, and confidence level.

In [ ]:
# List of ParseWT samples to process
samples = ['ParseWT21', 'ParseWT22', 'ParseWT33', 'ParseWT34', 'ParseWT35', 
           'ParseWT36', 'ParseWT37', 'ParseWT38', 'ParseWT39', 'ParseWT40']

for sample in samples:
    print(f"Processing {sample}...")
    
    # Read barcode translation file (CSV)
    barcode_df = pd.read_csv(f"/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/translated_{sample}_barcodes.csv", 
                            header=None,
                            skiprows=1,
                            names=['new_barcode','old_barcode'])
    
    # Read species assignments file (tab-delimited)
    species_df = pd.read_csv(f"/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/composite_bam2counts/{sample}/species.assignments", 
                            sep='\t',
                            header=None,
                            names=['new_barcode', 'species_predicted', 'doublet_singlet_score', 'confidence'])
    
    # Merge on new_barcode
    merged_df = pd.merge(barcode_df, species_df, on='new_barcode', how='outer')
    
    # Save result
    merged_df.to_csv(f"metadata_barcode_species_{sample}.csv", index=False)

print("All samples processed!")

### 7. Created list of barcodes of each species for each sublibrary.

In [2]:
samples = ['ParseWT21', 'ParseWT22', 'ParseWT33', 'ParseWT34', 'ParseWT35', 
           'ParseWT36', 'ParseWT37', 'ParseWT38', 'ParseWT39', 'ParseWT40']

meta = {}
for sample in samples:
    print(f"Processing {sample}...")
    meta[sample] = pd.read_csv(f"/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed/metadata_barcode_species_{sample}.csv")

Processing ParseWT21...
Processing ParseWT22...
Processing ParseWT33...
Processing ParseWT34...
Processing ParseWT35...
Processing ParseWT36...
Processing ParseWT37...
Processing ParseWT38...
Processing ParseWT39...
Processing ParseWT40...


In [3]:
meta_s = {}
for sample in meta:
    meta_s[sample] = meta[sample][meta[sample]["doublet_singlet_score"] == "S"]

In [4]:
for sample in meta:
    print(f"{sample} dimensions: {meta[sample].shape[0]} rows, {meta[sample].shape[1]} columns")

ParseWT21 dimensions: 878833 rows, 5 columns
ParseWT22 dimensions: 875617 rows, 5 columns
ParseWT33 dimensions: 874555 rows, 5 columns
ParseWT34 dimensions: 882171 rows, 5 columns
ParseWT35 dimensions: 882650 rows, 5 columns
ParseWT36 dimensions: 882809 rows, 5 columns
ParseWT37 dimensions: 883160 rows, 5 columns
ParseWT38 dimensions: 882919 rows, 5 columns
ParseWT39 dimensions: 883527 rows, 5 columns
ParseWT40 dimensions: 881903 rows, 5 columns


In [5]:
meta_human = {}
for sample in meta_s:
   meta_human[sample] = meta_s[sample][meta_s[sample]["species_predicted"] == "GRCh38"]

meta_gorila = {}
for sample in meta_s:
   meta_gorila[sample] = meta_s[sample][meta_s[sample]["species_predicted"] == "gorGor4"]

meta_chimp = {}
for sample in meta_s:
   meta_chimp[sample] = meta_s[sample][meta_s[sample]["species_predicted"] == "Pan-tro-3-0"]

meta_macaque = {}
for sample in meta_s:
   meta_macaque[sample] = meta_s[sample][meta_s[sample]["species_predicted"] == "Macaca-fascicularis-6-0"]

meta_mouse = {}
for sample in meta_s:
   meta_mouse[sample] = meta_s[sample][meta_s[sample]["species_predicted"] == "GRCm39"]

In [6]:
meta_species = {'human': meta_human, 'gorila': meta_gorila, 'chimp': meta_chimp, 'macaque': meta_macaque, 'mouse': meta_mouse}
for species_name, meta_dict in meta_species.items():
    print(f"\n{species_name.upper()}:")
    for sample in meta_dict:
        print(f"  {sample} dimensions: {meta_dict[sample].shape[0]} rows, {meta_dict[sample].shape[1]} columns")


HUMAN:
  ParseWT21 dimensions: 154885 rows, 5 columns
  ParseWT22 dimensions: 194135 rows, 5 columns
  ParseWT33 dimensions: 302092 rows, 5 columns
  ParseWT34 dimensions: 184830 rows, 5 columns
  ParseWT35 dimensions: 320893 rows, 5 columns
  ParseWT36 dimensions: 172378 rows, 5 columns
  ParseWT37 dimensions: 3996 rows, 5 columns
  ParseWT38 dimensions: 4090 rows, 5 columns
  ParseWT39 dimensions: 454671 rows, 5 columns
  ParseWT40 dimensions: 6853 rows, 5 columns

GORILA:
  ParseWT21 dimensions: 27113 rows, 5 columns
  ParseWT22 dimensions: 4455 rows, 5 columns
  ParseWT33 dimensions: 33719 rows, 5 columns
  ParseWT34 dimensions: 30260 rows, 5 columns
  ParseWT35 dimensions: 2147 rows, 5 columns
  ParseWT36 dimensions: 31857 rows, 5 columns
  ParseWT37 dimensions: 186270 rows, 5 columns
  ParseWT38 dimensions: 238043 rows, 5 columns
  ParseWT39 dimensions: 1520 rows, 5 columns
  ParseWT40 dimensions: 3126 rows, 5 columns

CHIMP:
  ParseWT21 dimensions: 37548 rows, 5 columns
  Parse

In [7]:
species_data = {
   'human': meta_human,
   'gorilla': meta_gorila,
   'chimp': meta_chimp,
   'macaque': meta_macaque,
   'mouse': meta_mouse
}

# Loop through each species
for species_name, species_meta in species_data.items():
   for sample in species_meta:
       barcodes = species_meta[sample]['new_barcode']
       barcodes.to_csv(f"{sample}_{species_name}_barcodes.csv", index=False, header=['new_barcode'])

### 8. Filtering FASTQ files into separate species. 

#### 8.1 Making lists of headers corresponding to each species per sublibarary. 

In [4]:
%%bash
nohup python -c "
import pysam
import csv
import glob
import os

# Define paths
barcode_dir = '/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed'
bam_dir = '/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/modify_bam/modified_bam'
output_dir = '/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed/Fastq_filters'

# Define sublibraries and species
sublibraries = ['ParseWT21', 'ParseWT22', 'ParseWT33', 'ParseWT34', 'ParseWT35', 
               'ParseWT36', 'ParseWT37', 'ParseWT38', 'ParseWT39', 'ParseWT40']
species = ['human', 'chimp', 'gorilla', 'macaque', 'mouse']

for sublibrary in sublibraries:
    print(f'Processing {sublibrary}...', flush=True)
    
    bam_file = f'{bam_dir}/translated_{sublibrary}.bam'
    
    for sp in species:
        print(f'  Processing {sp}...', flush=True)
        
        barcode_file = f'{barcode_dir}/{sublibrary}_{sp}_barcodes.csv'
        
        barcodes_to_find = set()
        with open(barcode_file, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                barcodes_to_find.add(row[0])
        
        matching_headers = []
        with pysam.AlignmentFile(bam_file, 'rb') as bam:
            for read in bam:
                if read.has_tag('CB') and read.get_tag('CB') in barcodes_to_find:
                    header = read.query_name
                    if '@' in header:
                        header = header.split('@')[1].split(',')[0]
                    matching_headers.append(header)
        
        output_file = f'{output_dir}/headers_{sp}_{sublibrary}.csv'
        with open(output_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['header'])
            for header in matching_headers:
                writer.writerow([header])
        
        print(f'    Found {len(matching_headers)} headers -> {output_file}', flush=True)

print('All processing complete!', flush=True)
" > bam_processing.log 2>&1 &

#### 8.2 Concatanting FASTQ of each sublibrary across sequencing lanes. 

In [ ]:
cd /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_merged
 
for i in {21..22}; do echo $i; cat SLX-25498.ParseWT$i.*.r_1.fq.gz > SLX-25498.ParseWT$i.22K75GLT4.combined.r_1.fq.gz& done

for i in {21..22}; do echo $i; cat SLX-25498.ParseWT$i.*.r_2.fq.gz > SLX-25498.ParseWT$i.22K75GLT4.combined.r_2.fq.gz& done

for i in {33..40}; do echo $i; cat SLX-25498.ParseWT$i.*.r_1.fq.gz > SLX-25498.ParseWT$i.22K75GLT4.combined.r_1.fq.gz& done

for i in {33..40}; do echo $i; cat SLX-25498.ParseWT$i.*.r_2.fq.gz > SLX-25498.ParseWT$i.22K75GLT4.combined.r_2.fq.gz& done

#### 8.3 Filtering the fastq with header lists. 

In [ ]:
#saved as filter_merged_fastq.py at /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered/filter_merged_fastq.py
import os
import csv
import gzip
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import cpu_count

def load_headers_from_csv(csv_file):
    """Load headers from CSV into a set for fast lookup"""
    headers = set()
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header row
        for row in reader:
            headers.add(row[0])
    return headers

def filter_fastq_gz(input_fastq, output_fastq, headers_to_keep):
    """Filter compressed FASTQ file keeping only reads with headers in the set"""
    kept_reads = 0
    
    with gzip.open(input_fastq, 'rt') as infile, gzip.open(output_fastq, 'wt') as outfile:
        while True:
            # Read FASTQ record (4 lines)
            header = infile.readline().strip()
            if not header:  # End of file
                break
            sequence = infile.readline().strip()
            plus = infile.readline().strip()
            quality = infile.readline().strip()
            
            # Extract read ID from header (remove @ and everything after first space)
            read_id = header[1:].split()[0]
            
            # Keep read if header matches (saves as FASTQ format with quality scores)
            if read_id in headers_to_keep:
                outfile.write(f"{header}\n{sequence}\n{plus}\n{quality}\n")
                kept_reads += 1
    
    return kept_reads

def process_single_task(task):
    """Process a single FASTQ filtering task"""
    fastq_file, header_file, output_file, sublibrary, species, read_type = task
    
    try:
        # Load headers INSIDE the worker process to avoid memory duplication
        headers_to_keep = load_headers_from_csv(header_file)
        
        # Filter FASTQ
        kept_reads = filter_fastq_gz(fastq_file, output_file, headers_to_keep)
        
        # Clear headers from memory immediately
        del headers_to_keep
        
        return {
            'sublibrary': sublibrary,
            'species': species,
            'read_type': read_type,
            'output_file': os.path.basename(output_file),
            'kept_reads': kept_reads,
            'status': 'success'
        }
    except Exception as e:
        return {
            'sublibrary': sublibrary,
            'species': species,
            'read_type': read_type,
            'error': str(e),
            'status': 'error'
        }

def main():
    print("Starting FASTQ filtering job...")
    
    # Configuration
    merged_fastq_dir = "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_merged"
    header_dir = "/cephfs2/hannas/MultiSpeciesComp/final/analysis/demultiplexing_cellbouncer/demultiplexed/Fastq_filters"
    output_dir = "/cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered"
    
    # Use fewer workers to reduce memory pressure
    max_workers = min(int(os.environ.get('SLURM_CPUS_PER_TASK', cpu_count())), 8)
    
    print(f"Output directory: {output_dir}")
    print(f"Using {max_workers} parallel workers (reduced for memory)")
    
    os.makedirs(output_dir, exist_ok=True)
    
    sublibraries = ["ParseWT21", "ParseWT22", "ParseWT33", "ParseWT34", "ParseWT35", 
                   "ParseWT36", "ParseWT37", "ParseWT38", "ParseWT39", "ParseWT40"]
    species = ["human", "chimp", "gorilla", "macaque", "mouse"]
    
    # Build task list
    tasks = []
    for sublibrary in sublibraries:
        for sp in species:
            # Check if header file exists
            header_file = f"{header_dir}/headers_{sp}_{sublibrary}.csv"
            if not os.path.exists(header_file):
                print(f"Skipping {sublibrary} {sp}: header file not found - {header_file}")
                continue
            
            # Process R1 file - updated filename format
            r1_file = f"{merged_fastq_dir}/SLX-25498.{sublibrary}.22K75GLT4.combined.r_1.fq.gz"
            if os.path.exists(r1_file):
                output_r1 = f"{output_dir}/{sublibrary}_{sp}_R1.fq.gz"
                tasks.append((r1_file, header_file, output_r1, sublibrary, sp, "R1"))
            else:
                print(f"R1 file not found: {r1_file}")
            
            # Process R2 file - updated filename format
            r2_file = f"{merged_fastq_dir}/SLX-25498.{sublibrary}.22K75GLT4.combined.r_2.fq.gz"
            if os.path.exists(r2_file):
                output_r2 = f"{output_dir}/{sublibrary}_{sp}_R2.fq.gz"
                tasks.append((r2_file, header_file, output_r2, sublibrary, sp, "R2"))
            else:
                print(f"R2 file not found: {r2_file}")
    
    print(f"Found {len(tasks)} filtering tasks to process")
    
    if not tasks:
        print("No tasks found. Check file paths and header files.")
        return
    
    # Process tasks in parallel with reduced concurrency
    completed = 0
    total_reads = 0
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        future_to_task = {executor.submit(process_single_task, task): task for task in tasks}
        
        for future in as_completed(future_to_task):
            result = future.result()
            completed += 1
            
            if result['status'] == 'success':
                total_reads += result['kept_reads']
                print(f"[{completed}/{len(tasks)}] {result['sublibrary']} {result['species']} "
                      f"{result['read_type']}: {result['kept_reads']} reads -> {result['output_file']}")
            else:
                print(f"[{completed}/{len(tasks)}] ERROR {result['sublibrary']} {result['species']} "
                      f"{result['read_type']}: {result['error']}")
    
    print(f"\nAll filtering complete!")
    print(f"Total reads kept: {total_reads}")
    print(f"Files saved to: {output_dir}")

if __name__ == "__main__":
    main()

In [ ]:
#!/bin/bash
#SBATCH --job-name=filter_fastq
#SBATCH --mem=128G
#SBATCH --cpus-per-task=16
#SBATCH --time=12:00:00
#SBATCH --output=filter_fastq_%j.out
#SBATCH --error=filter_fastq_%j.err

# Load Python if needed (adjust for your cluster)
# module load python/3.9  # Uncomment if needed

echo "Starting FASTQ filtering job at: $(date)"
echo "Job ID: $SLURM_JOB_ID"
echo "CPUs allocated: $SLURM_CPUS_PER_TASK"
echo "Memory allocated: $SLURM_MEM_PER_NODE MB"

# Change to the script directory
cd /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered

# Run the Python script
python3 filter_merged_fastq.py

echo "Job completed at: $(date)"

### 9. Realignment of species FASTQ to species-specific reference genomes using ParseNIP. 

In [ ]:
#runing split-pipe thoutgh ParseNIP
nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
-profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
-genome homo_sapiens.GRCh38.release_102 --genome_name homo_sapiens.GRCh38.release_102 \
--samp_list /cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/SampleLoadingTable_MultiSpieciesComp.txt \
--fastq /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered/human --chemistry v1 -bg

nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
-profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
-genome pan_troglodytes.Pan_tro_3.0.release_105 --genome_name pan_troglodytes.Pan_tro_3.0.release_105 \
--samp_list /cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/SampleLoadingTable_MultiSpieciesComp.txt \
--fastq /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered/chimp --chemistry v1 -bg

nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
-profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
-genome gorilla_gorilla.gorGor4.release_110 --genome_name gorilla_gorilla.gorGor4.release_110 \
    --samp_list /cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/SampleLoadingTable_MultiSpieciesComp.txt \
--fastq /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered/gorilla --chemistry v1 -bg

nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
-profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
-genome macaca_fascicularis.Macaca_fascicularis_6.0.release_112 --genome_name macaca_fascicularis.Macaca_fascicularis_6.0.release_112 \
--samp_list /cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/SampleLoadingTable_MultiSpieciesComp.txt \
--fastq /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered/macaque --chemistry v1 -bg

nextflow run -config /public/singularity/containers/nextflow/lmb-nextflow/genomes.config,/public/singularity/containers/nextflow/ParseNIP/nextflow.config \
-profile lmb_cluster /public/singularity/containers/nextflow/ParseNIP/main.nf \
-genome mus_musculus.GRCm39.release_108 --genome_name mus_musculus.GRCm39.release_108 \
--samp_list /cephfs2/hannas/MultiSpeciesComp/final/expdata/Parsnip_mixedref_v2/SampleLoadingTable_MultiSpieciesComp.txt \
--fastq /cephfs2/hannas/MultiSpeciesComp/final/expdata/Fastq_filtered/mouse --chemistry v1 -bgbg

### 10. CrossFilt removal of annotation biases from resulting BAM files: crossfilt-lift.

CrossFilt remove the sequences do nto reciprcally match between the species. 
It alignes reads from target bam to the target genome (the species we are converting from). 
Next, each nucleotide in the read that matches the target genome is replaced with the corresponding nucleotide from the query genome, accounting for insertions and deletions.

In [ ]:
cd /cephfs2/hannas/MultiSpeciesComp/final/genome/CrossFilt
conda activate crossfilt

#getign chain files from USCS 
wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToPanTro6.over.chain.gz
wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToGorGor5.over.chain.gz
wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToMacFas5.over.chain.gz
wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/liftOver/hg38ToMm39.over.chain.gz

#crossfilt-lift commands
CHAIN_dir = '/cephfs2/hannas/MultiSpeciesComp/final/genome/CrossFilt/chain_files'
ENSEMBLE_dir = '/cephfs2/hannas/MultiSpeciesComp/final/genome/Esemble_genomes'

crossfilt-lift -i INPUT -o 'crossfilt_chimp2human' -c $CHAIN_dir/hg38ToPanTro6.over.chain.gz -t $ENSEMBLE_dir/pan_troglodytes/Pan_tro_3.0/Release_105/FASTA/pan_troglodytes__Pan_tro_3.0__release105.dna.fa -q $ENSEMBLE_dir/homo_sapiens/GRCh38/Release_102/FASTA/homo_sapiens__GRCh38__release102.dna.fa -p
crossfilt-lift -i INPUT -o 'crossfilt_gorila2human' -c $CHAIN_dir/hg38ToGorGor5.over.chain.gz -t $ENSEMBLE_dir/gorilla_gorilla/gorGor4/Release_110/FASTA/gorilla_gorilla__gorGor4__release110.dna.fa -q $ENSEMBLE_dir/homo_sapiens/GRCh38/Release_102/FASTA/homo_sapiens__GRCh38__release102.dna.fa -p
crossfilt-lift -i INPUT -o 'crossfilt_macaque2human' -c $CHAIN_dir/hg38ToMacFas5.over.chain.gz -t $ENSEMBLE_dir/macaca_mulatta/Mmul_10/Release_105/FASTA/macaca_mulatta__Mmul_10__release105.dna.fa -q $ENSEMBLE_dir/homo_sapiens/GRCh38/Release_102/FASTA/homo_sapiens__GRCh38__release102.dna.fa -p
crossfilt-lift -i INPUT -o 'crossfilt_mouse2human' -c $CHAIN_dir/hg38ToMm39.over.chain.gz -t $ENSEMBLE_dir/mus_musculus/GRCm39/Release_108/FASTA/mus_musculus__GRCm39__release108.dna.fa -q $ENSEMBLE_dir/homo_sapiens/GRCh38/Release_102/FASTA/homo_sapiens__GRCh38__release102.dna.fa -p